In [ ]:
%load_ext autoreload
%autoreload 2

## TODO
- Generate periodic 2d mesh
- [x] Write as GMSH (gmsh22)
- Use pygalmesh with contour instead of points/edges
- Pygalmesh multi-domain for 2d mesh? (i.e. 0==air, 1==Si, 2==?? element
- Add cropping method (select one pore)
- [x] Use k-means for distributing points in area
- [x] Add subroutine to calculate density of points for kmeans
- Generate inside mesh as well as outside mesh in same function
- Implement periodicity by tiling image with a wrap-around buffer (i.e. tile 3x3 -> triangulate -> and pull out the middle)
- Close contours and use subdivision algorithm to generate edge points

In [ ]:
from nanomesh.volume import Volume
from nanomesh.plane import Plane

In [ ]:
vol = Volume.load('sample_data.npy')

In [ ]:
vol.show_slice(along='x')

In [ ]:
plane = vol.select_plane(x=161)

In [ ]:
plane.show()

In [ ]:
from skimage import filters

In [ ]:
plane_gauss = plane.apply_np(filters.gaussian, sigma=5)
plane_gauss.show()

In [ ]:
filters.try_all_threshold(plane_gauss.array_view, figsize=(10, 15), verbose=True)

In [ ]:
thresh = filters.threshold_li(plane_gauss.array_view)

seg = Plane.from_array(1.0 * (plane_gauss.array_view >= thresh))
seg.show()

In [ ]:
mesh = seg.generate_mesh(pad=True, point_density=1/100, max_contour_dist=4, plot=True)

In [ ]:
import pyvista as pv
pv.plot_itk(mesh)

In [ ]:
mesh.write("plane.msh", file_format='gmsh22')

## Using pygalmesh

In [ ]:
import pygalmesh

points = np.vstack(contours)

index = 0
constraints = []
for contour in contours:
    n = len(contour) - 2
    contour_constraints = [[index+k, index+k + 1] for k in range(n)] + [[index+n, index]]
    index += len(contour_constraints)
    constraints.extend(contour_constraints)
    break

print(constraints)

mesh = pygalmesh.generate_2d(
    points=points,
    constraints=constraints,
    max_edge_size=10,
    num_lloyd_steps=0,
    B=1.4,
)

# pv.plot_itk(mesh)

plt.figure(figsize=(12, 12))

ax = plt.subplot(223)
ax.set_title(f'All triangles ({len(triangles)})')
ax.imshow(image, cmap='gray')
ax.triplot(*mesh.points.T[::-1], mesh.cells[0].data)

## Optimize mesh

In [ ]:
import optimesh
import meshplex
meshp = meshplex.Mesh(mesh.points, mesh.cells[0].data)
new = optimesh.optimize(meshp, method="CVT (block-diagonal)", tol=1.0e-5, max_num_steps=100)

In [ ]:
import pygmsh

In [ ]:
import pygmsh

with pygmsh.geo.Geometry() as geom:
    geom.add_rectangle(0, 1, 0, 1, z=1)
    
    
    mesh = geom.generate_mesh()

In [ ]:
pv.plot_itk(mesh)

In [ ]:
# meshio.write_points_cells("in.vtu", verts, [("triangle", faces)])
import pygalmesh

# mesh.write('in.vtu')

# mesh = pygalmesh.remesh_surface(
#     "in.vtu",
#     max_edge_size_at_feature_edges=0.025,
#     min_facet_angle=25,
#     max_radius_surface_delaunay_ball=0.1,
#     max_facet_distance=0.001,
#     verbose=True,
# )